In [1]:
import warnings

import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pingouin as pg

warnings.filterwarnings("ignore")

In [2]:
# set seed to make the results fully reproducible
seed: int = sum(map(ord, "specification_error"))
rng: np.random.Generator = np.random.default_rng(seed=seed)

az.style.use("arviz-darkgrid")
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.facecolor"] = "white"

%config InlineBackend.figure_format = "retina"

In [3]:
# Generiamo dati casuali
np.random.seed(42)
n = 100  # Numero di osservazioni

# Variabili indipendenti con correlazione negativa tra loro
motivazione = np.random.normal(100, 10, n)
ansia = 200 + 0.75 * motivazione + np.random.normal(0, 5, n)

# Variabile dipendente, con peso maggiore sulla motivazione rispetto all'ansia
prestazione = 5 * motivazione - 1 * ansia + np.random.normal(0, 50, n)

# Creazione DataFrame
data = pd.DataFrame(
    {"Motivazione": motivazione, "Ansia": ansia, "Prestazione": prestazione}
)

In [4]:
model_full = bmb.Model("Prestazione ~ Motivazione + Ansia", data=data)
results_full = model_full.fit(nuts_sampler="numpyro")

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [5]:
az.summary(results_full, round_to=2)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Ansia,-1.11,1.12,-3.25,0.88,0.02,0.02,2105.71,2604.24,1.0
Intercept,-87.85,243.86,-556.95,355.76,4.87,3.44,2503.36,2737.24,1.0
Motivazione,6.23,0.95,4.37,7.95,0.02,0.01,2034.55,2237.52,1.0
Prestazione_sigma,54.30,3.81,47.15,61.22,0.06,0.04,3769.41,2829.23,1.0


In [6]:
# Analisi di regressione con pingouin
results_full = pg.linear_regression(data[["Motivazione", "Ansia"]], data["Prestazione"])
results_full

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,-84.058695,244.261908,-0.344133,7.314908e-01,0.467656,0.456679,-568.850966,400.733576
1,Motivazione,6.222523,0.977770,6.363994,6.510176e-09,0.467656,0.456679,4.281920,8.163125
2,Ansia,-1.122768,1.143817,-0.981597,3.287403e-01,0.467656,0.456679,-3.392928,1.147393


In [7]:
model_ansia_only = bmb.Model("Prestazione ~ Ansia", data=data)
results_ansia_only = model_ansia_only.fit(nuts_sampler="numpyro")

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [8]:
az.summary(results_ansia_only, round_to=2)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Ansia,4.65,0.84,3.15,6.32,0.01,0.01,4138.00,2886.12,1.0
Intercept,-1050.91,230.73,-1500.45,-631.77,3.62,2.72,4100.74,2885.36,1.0
Prestazione_sigma,64.20,4.53,56.46,73.56,0.07,0.05,4091.20,3220.06,1.0


In [9]:
results_ansia_only = pg.linear_regression(data[["Ansia"]], data["Prestazione"])
results_ansia_only

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,-1052.984249,226.249178,-4.654091,1.020933e-05,0.245386,0.237686,-1501.968379,-604.000119
1,Ansia,4.653853,0.824399,5.645148,1.608208e-07,0.245386,0.237686,3.017861,6.289846


In [10]:
%load_ext watermark
%watermark -n -u -v -iv -w -m 

Last updated: Sun Jun 16 2024

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.25.0

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.4.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

arviz     : 0.18.0
bambi     : 0.13.0
pandas    : 2.2.2
numpy     : 1.26.4
matplotlib: 3.8.4
pingouin  : 0.5.4

Watermark: 2.4.3

